# tests

Imports

In [3]:
# ...

#### Config: Paths

In [13]:
# path_root = './canto-01-long'
path_root = './canto-01-short'
path_mh_txt = os.path.join(path_root, '_weight_mh.txt')
path_mh_csv = os.path.join(path_root, '_weight_mh.csv')
path_blends_txt = os.path.join(path_root, '_weight_blends.txt')
path_blends_csv = os.path.join(path_root, '_weight_blends.csv')
path_usda = os.path.join(path_root, 'mh_out.usda')


## Build CSV weight files

Imports

In [2]:
import os
import re
import json
import math

import pandas as pd

Code

In [25]:

def save_weights_as_csv(path_input: str, path_output: str, max_lines: int = None) -> None:

    try:
        
        file_in = open(path_input, 'r')
        max_lines = max_lines or math.inf
        df: pd.DataFrame = None

        i = -1
        for line in file_in:
            i = i + 1

            # line = re.sub(r"^(.*\d+\: )?(\[.+\])(.*)$", r"\2", line)
            line = re.sub(r"^([^\[]*(\d+\:)?[^\[]*)(\[.+\])(.*)$", r"\3", line)
            # print(line)
            # return
            weights = json.loads(line)
            if i == 0:
                df = pd.DataFrame(columns=[i for i in range(len(weights))])

            df.loc[i] = weights
            if i == max_lines - 1:
                break
        
        df.to_csv(path_output, index=False)
    
    finally:
        if file_in:
            file_in.close()

# Parse meta human curves
# save_weights_as_csv(path_input=path_mh_txt, path_output=path_mh_csv, max_lines=None)

# # Parse blendshapes
save_weights_as_csv(path_input=path_blends_txt, path_output=path_blends_csv, max_lines=None)

## Regressor: Blendshape $⟶$ Meta human

Imports

In [2]:
from sklearn.linear_model import LinearRegression

import numpy as np
import pandas as pd

### Load Data

#### From `.csv` files

In [16]:

df_blends = pd.read_csv(path_blends_csv)
df_mh = pd.read_csv(path_mh_csv)

weights_blends = df_blends.values
weights_mh = df_mh.values
# df_mh.describe()

#### Raw (from `.usd`)

In [4]:
names_mh_curves = ["CTRL_expressions_browDownL", "CTRL_expressions_browDownR", "CTRL_expressions_browLateralL", "CTRL_expressions_browLateralR", "CTRL_expressions_browRaiseInL", "CTRL_expressions_browRaiseInR", "CTRL_expressions_browRaiseOuterL", "CTRL_expressions_browRaiseOuterR", "CTRL_expressions_eyeBlinkL", "CTRL_expressions_eyeBlinkR", "CTRL_expressions_eyeWidenL", "CTRL_expressions_eyeWidenR", "CTRL_expressions_eyeSquintInnerL", "CTRL_expressions_eyeSquintInnerR", "CTRL_expressions_eyeCheekRaiseL", "CTRL_expressions_eyeCheekRaiseR", "CTRL_expressions_eyeLookUpL", "CTRL_expressions_eyeLookUpR", "CTRL_expressions_eyeLookDownL", "CTRL_expressions_eyeLookDownR", "CTRL_expressions_eyeLookLeftL", "CTRL_expressions_eyeLookLeftR", "CTRL_expressions_eyeLookRightL", "CTRL_expressions_eyeLookRightR", "CTRL_expressions_noseWrinkleL", "CTRL_expressions_noseWrinkleR", "CTRL_expressions_mouthCheekBlowL", "CTRL_expressions_mouthCheekBlowR", "CTRL_expressions_mouthLipsBlowL", "CTRL_expressions_mouthLipsBlowR", "CTRL_expressions_mouthLeft", "CTRL_expressions_mouthRight", "CTRL_expressions_mouthUpperLipRaiseL", "CTRL_expressions_mouthUpperLipRaiseR", "CTRL_expressions_mouthLowerLipDepressL", "CTRL_expressions_mouthLowerLipDepressR", "CTRL_expressions_mouthCornerPullL", "CTRL_expressions_mouthCornerPullR", "CTRL_expressions_mouthStretchL", "CTRL_expressions_mouthStretchR", "CTRL_expressions_mouthDimpleL", "CTRL_expressions_mouthDimpleR", "CTRL_expressions_mouthCornerDepressL", "CTRL_expressions_mouthCornerDepressR", "CTRL_expressions_mouthPressUL", "CTRL_expressions_mouthPressUR", "CTRL_expressions_mouthPressDL", "CTRL_expressions_mouthPressDR", "CTRL_expressions_mouthLipsPurseUL", "CTRL_expressions_mouthLipsPurseUR", "CTRL_expressions_mouthLipsPurseDL", "CTRL_expressions_mouthLipsPurseDR", "CTRL_expressions_mouthLipsTowardsUL", "CTRL_expressions_mouthLipsTowardsUR", "CTRL_expressions_mouthLipsTowardsDL", "CTRL_expressions_mouthLipsTowardsDR", "CTRL_expressions_mouthFunnelUL", "CTRL_expressions_mouthFunnelUR", "CTRL_expressions_mouthFunnelDL", "CTRL_expressions_mouthFunnelDR", "CTRL_expressions_mouthLipsTogetherUL", "CTRL_expressions_mouthLipsTogetherUR", "CTRL_expressions_mouthLipsTogetherDL", "CTRL_expressions_mouthLipsTogetherDR", "CTRL_expressions_mouthUpperLipRollInL", "CTRL_expressions_mouthUpperLipRollInR", "CTRL_expressions_mouthLowerLipRollInL", "CTRL_expressions_mouthLowerLipRollInR", "CTRL_expressions_jawOpen", "CTRL_expressions_jawLeft", "CTRL_expressions_jawRight", "CTRL_expressions_jawFwd", "CTRL_expressions_jawChinRaiseDL", "CTRL_expressions_jawChinRaiseDR"]

names_mh_curves = [name.replace('CTRL_expressions_', '') for name in names_mh_curves]


names_blends = ["_blendShape1_browLowerL", "_blendShape1_browLowerR", "_blendShape1_innerBrowRaiserL", "_blendShape1_innerBrowRaiserR", "_blendShape1_outerBrowRaiserL", "_blendShape1_outerBrowRaiserR", "_blendShape1_eyesLookLeft", "_blendShape1_eyesLookRight", "_blendShape1_eyesLookUp", "_blendShape1_eyesLookDown", "_blendShape1_eyesCloseL", "_blendShape1_eyesCloseR", "_blendShape1_eyesUpperLidRaiserL", "_blendShape1_eyesUpperLidRaiserR", "_blendShape1_squintL", "_blendShape1_squintR", "_blendShape1_cheekRaiserL", "_blendShape1_cheekRaiserR", "_blendShape1_cheekPuffL", "_blendShape1_cheekPuffR", "_blendShape1_noseWrinklerL", "_blendShape1_noseWrinklerR", "_blendShape1_jawDrop", "_blendShape1_jawDropLipTowards", "_blendShape1_jawThrust", "_blendShape1_jawSlideLeft", "_blendShape1_jawSlideRight", "_blendShape1_mouthSlideLeft", "_blendShape1_mouthSlideRight", "_blendShape1_dimplerL", "_blendShape1_dimplerR", "_blendShape1_lipCornerPullerL", "_blendShape1_lipCornerPullerR", "_blendShape1_lipCornerDepressorL", "_blendShape1_lipCornerDepressorR", "_blendShape1_lipStretcherL", "_blendShape1_lipStretcherR", "_blendShape1_upperLipRaiserL", "_blendShape1_upperLipRaiserR", "_blendShape1_lowerLipDepressorL", "_blendShape1_lowerLipDepressorR", "_blendShape1_chinRaiser", "_blendShape1_lipPressor", "_blendShape1_pucker", "_blendShape1_funneler", "_blendShape1_lipSuck"]

names_blends = [name.replace('_blendShape1_', '') for name in names_blends]

weights_blends = np.array([
    [0, 0, 0.5275745, 0.42562675, 0.11813607, 0.072490655, 0.08326936, 0.046086557, 0, 0.02894416, 0, 0, 0.06632169, 0.24260138, 0, 0, 0, 0, 0, 0, 0, 0, 0.22479105, 0, 0, 0, 0.12671344, 0, 0.014078964, 0.2096956, 0.78019685, 0.14192654, 0.1716196, 0, 0.43281862, 0.1250949, 0.01650266, 0, 0, 0, 0, 0, 0, 0, 0, 0.12189539],
    [0, 0, 0.5316879, 0.4296941, 0.11489728, 0.07314639, 0.08050206, 0.043806277, 0, 0.032465003, 0, 0, 0.07041683, 0.24656239, 0, 0, 0, 0, 0, 0, 0, 0, 0.16560663, 0, 0, 0, 0.10248774, 0, 0.010578315, 0.23960285, 0.8272298, 0.18687817, 0.21114767, 0, 0.36097357, 0.13410169, 0.029525725, 0, 0, 0, 0, 0.06144752, 0, 0, 0, 0.12735894],
    [0, 0, 0.53545934, 0.43217716, 0.10709738, 0.0753164, 0.075476944, 0.039058927, 0, 0.035691027, 0, 0, 0.081670724, 0.2567328, 0, 0, 0, 0, 0, 0, 0, 0, 0.11850466, 0, 0, 0, 0.07326623, 0, 0.025137186, 0.26291668, 0.8826134, 0.24104786, 0.23183846, 0, 0.15717763, 0.11752446, 0.012616659, 0, 0, 0, 0, 0.18398458, 0, 0, 0, 0.10277637],
    [0, 0, 0.5367419, 0.4266354, 0.10231372, 0.07804342, 0.07206717, 0.03504996, 0, 0.03419071, 0, 0, 0.09337057, 0.26721916, 0, 0, 0, 0, 0, 0, 0, 0, 0.10265807, 0, 0, 0, 0.07502933, 0, 0.03282166, 0.25086713, 0.86872405, 0.26660776, 0.23527455, 0, 0, 0.079189174, 0, 0, -0, 0, 0, 0.21208611, 0, 0, 0.01938673, 0.082280524],
    [0, 0, 0.53729284, 0.41302875, 0.10849441, 0.0822979, 0.07280974, 0.035206437, 0, 0.025201678, 0, 0, 0.098639295, 0.27185127, 0, 0, 0, 0, 0, 0, 0, 0, 0.14382224, 0, 0, 0, 0.08306411, 0, 0.03281624, 0.25467917, 0.78497314, 0.23584178, 0.1914369, 0, 0, 0.016126905, 0, 0, -0, 0, 0, 0.1283324, 0, 0, 0.0104871495, 0.079502165],
    [0, 0, 0.5393888, 0.40201843, 0.12024226, 0.08416013, 0.07653246, 0.038072076, 0, 0.016718155, 0, 0, 0.09676899, 0.26920432, 0, 0, 0, 0, 0, 0, 0, 0, 0.1875947, 0, 0, 0, 0.09925031, 0, 0.024456928, 0.2656719, 0.75592023, 0.20279185, 0.1591677, 0, 0.00939027, 0, 0, 0, 0.013222429, 0, 0, 0.025483394, 0, 0, 0, 0.09050906],
])

weights_mh = np.array([
    [0, 0, 0.5275745, 0.42562675, 0.5275745, 0.42562675, 0.11813607, 0.072490655, 0, 0, 0.06632169, 0.24260138, 0, 0, 0, 0, 0, 0, 0.02894416, 0.02894416, 0.08326936, 0.08326936, 0.046086557, 0.046086557, 0, 0, 0, 0, 0, 0, 0, 0.014078964, 0, 0, 0, 0, 0.14192654, 0.1716196, 0.1250949, 0.01650266, 0.2096956, 0.78019685, 0, 0.43281862, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.12189539, 0.12189539, 0.12189539, 0.12189539, 0.22479105, 0, 0.12671344, 0, 0, 0],            
    [0, 0, 0.5316879, 0.4296941, 0.5316879, 0.4296941, 0.11489728, 0.07314639, 0, 0, 0.07041683, 0.24656239, 0, 0, 0, 0, 0, 0, 0.032465003, 0.032465003, 0.08050206, 0.08050206, 0.043806277, 0.043806277, 0, 0, 0, 0, 0, 0, 0, 0.010578315, 0, 0, 0, 0, 0.18687817, 0.21114767, 0.13410169, 0.029525725, 0.23960285, 0.8272298, 0, 0.36097357, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.12735894, 0.12735894, 0.12735894, 0.12735894, 0.16560663, 0, 0.10248774, 0, 0.06144752, 0.06144752],
    [0, 0, 0.53545934, 0.43217716, 0.53545934, 0.43217716, 0.10709738, 0.0753164, 0, 0, 0.081670724, 0.2567328, 0, 0, 0, 0, 0, 0, 0.035691027, 0.035691027, 0.075476944, 0.075476944, 0.039058927, 0.039058927, 0, 0, 0, 0, 0, 0, 0, 0.025137186, 0, 0, 0, 0, 0.24104786, 0.23183846, 0.11752446, 0.012616659, 0.26291668, 0.8826134, 0, 0.15717763, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.10277637, 0.10277637, 0.10277637, 0.10277637, 0.11850466, 0, 0.07326623, 0, 0.18398458, 0.18398458],
    [0, 0, 0.5367419, 0.4266354, 0.5367419, 0.4266354, 0.10231372, 0.07804342, 0, 0, 0.09337057, 0.26721916, 0, 0, 0, 0, 0, 0, 0.03419071, 0.03419071, 0.07206717, 0.07206717, 0.03504996, 0.03504996, 0, 0, 0, 0, 0, 0, 0, 0.03282166, 0, 0, 0, 0, 0.26660776, 0.23527455, 0.079189174, 0, 0.25086713, 0.86872405, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.01938673, 0.01938673, 0.01938673, 0.01938673, 0, 0, 0, 0, 0.082280524, 0.082280524, 0.082280524, 0.082280524, 0.10265807, 0, 0.07502933, 0, 0.21208611, 0.21208611],
    [0, 0, 0.53729284, 0.41302875, 0.53729284, 0.41302875, 0.10849441, 0.0822979, 0, 0, 0.098639295, 0.27185127, 0, 0, 0, 0, 0, 0, 0.025201678, 0.025201678, 0.07280974, 0.07280974, 0.035206437, 0.035206437, 0, 0, 0, 0, 0, 0, 0, 0.03281624, 0, 0, 0, 0, 0.23584178, 0.1914369, 0.016126905, 0, 0.25467917, 0.78497314, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0104871495, 0.0104871495, 0.0104871495, 0.0104871495, 0, 0, 0, 0, 0.079502165, 0.079502165, 0.079502165, 0.079502165, 0.14382224, 0, 0.08306411, 0, 0.1283324, 0.1283324],
    [0, 0, 0.5393888, 0.40201843, 0.5393888, 0.40201843, 0.12024226, 0.08416013, 0, 0, 0.09676899, 0.26920432, 0, 0, 0, 0, 0, 0, 0.016718155, 0.016718155, 0.07653246, 0.07653246, 0.038072076, 0.038072076, 0, 0, 0, 0, 0, 0, 0, 0.024456928, 0, 0.013222429, 0, 0, 0.20279185, 0.1591677, 0, 0, 0.2656719, 0.75592023, 0, 0.00939027, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.09050906, 0.09050906, 0.09050906, 0.09050906, 0.1875947, 0, 0.09925031, 0, 0.025483394, 0.025483394],
])


### Build regressor

In [5]:
regressor = LinearRegression().fit(weights_blends, weights_mh)


# n_cols = 15
# df = pd.DataFrame(columns=[i for i in range(n_cols)])
# for i in range(30):
#     df.loc[i] = weights_mh[i, 0:15]

# df2 = df[:, [0:15]]

## Build `.usda`

#### `get_usda_str()`

In [36]:

def get_usda_str(prim_xform: str, prim_skew_anim: str, n_frames: int, frame_rate: int, mh_curveValues: list) -> str:
    '''
        TODO: 2022-11-17 - ADD Description
    '''
    usda_str = f'''
#usda 1.0
(
    defaultPrim = "{prim_xform}"
    endTimeCode = {n_frames - 1} 
    framesPerSecond = {frame_rate} 
    metersPerUnit = 0.01
    startTimeCode = 0
    timeCodesPerSecond = {frame_rate} 
)

def Xform "World"
{{
    def SkelAnimation "{prim_skew_anim}"
    {{
        token[] custom:mh_curveNames = ["CTRL_expressions_browDownL", "CTRL_expressions_browDownR", "CTRL_expressions_browLateralL", "CTRL_expressions_browLateralR", "CTRL_expressions_browRaiseInL", "CTRL_expressions_browRaiseInR", "CTRL_expressions_browRaiseOuterL", "CTRL_expressions_browRaiseOuterR", "CTRL_expressions_eyeBlinkL", "CTRL_expressions_eyeBlinkR", "CTRL_expressions_eyeWidenL", "CTRL_expressions_eyeWidenR", "CTRL_expressions_eyeSquintInnerL", "CTRL_expressions_eyeSquintInnerR", "CTRL_expressions_eyeCheekRaiseL", "CTRL_expressions_eyeCheekRaiseR", "CTRL_expressions_eyeLookUpL", "CTRL_expressions_eyeLookUpR", "CTRL_expressions_eyeLookDownL", "CTRL_expressions_eyeLookDownR", "CTRL_expressions_eyeLookLeftL", "CTRL_expressions_eyeLookLeftR", "CTRL_expressions_eyeLookRightL", "CTRL_expressions_eyeLookRightR", "CTRL_expressions_noseWrinkleL", "CTRL_expressions_noseWrinkleR", "CTRL_expressions_mouthCheekBlowL", "CTRL_expressions_mouthCheekBlowR", "CTRL_expressions_mouthLipsBlowL", "CTRL_expressions_mouthLipsBlowR", "CTRL_expressions_mouthLeft", "CTRL_expressions_mouthRight", "CTRL_expressions_mouthUpperLipRaiseL", "CTRL_expressions_mouthUpperLipRaiseR", "CTRL_expressions_mouthLowerLipDepressL", "CTRL_expressions_mouthLowerLipDepressR", "CTRL_expressions_mouthCornerPullL", "CTRL_expressions_mouthCornerPullR", "CTRL_expressions_mouthStretchL", "CTRL_expressions_mouthStretchR", "CTRL_expressions_mouthDimpleL", "CTRL_expressions_mouthDimpleR", "CTRL_expressions_mouthCornerDepressL", "CTRL_expressions_mouthCornerDepressR", "CTRL_expressions_mouthPressUL", "CTRL_expressions_mouthPressUR", "CTRL_expressions_mouthPressDL", "CTRL_expressions_mouthPressDR", "CTRL_expressions_mouthLipsPurseUL", "CTRL_expressions_mouthLipsPurseUR", "CTRL_expressions_mouthLipsPurseDL", "CTRL_expressions_mouthLipsPurseDR", "CTRL_expressions_mouthLipsTowardsUL", "CTRL_expressions_mouthLipsTowardsUR", "CTRL_expressions_mouthLipsTowardsDL", "CTRL_expressions_mouthLipsTowardsDR", "CTRL_expressions_mouthFunnelUL", "CTRL_expressions_mouthFunnelUR", "CTRL_expressions_mouthFunnelDL", "CTRL_expressions_mouthFunnelDR", "CTRL_expressions_mouthLipsTogetherUL", "CTRL_expressions_mouthLipsTogetherUR", "CTRL_expressions_mouthLipsTogetherDL", "CTRL_expressions_mouthLipsTogetherDR", "CTRL_expressions_mouthUpperLipRollInL", "CTRL_expressions_mouthUpperLipRollInR", "CTRL_expressions_mouthLowerLipRollInL", "CTRL_expressions_mouthLowerLipRollInR", "CTRL_expressions_jawOpen", "CTRL_expressions_jawLeft", "CTRL_expressions_jawRight", "CTRL_expressions_jawFwd", "CTRL_expressions_jawChinRaiseDL", "CTRL_expressions_jawChinRaiseDR"]

        float[] custom:mh_curveValues.timeSamples = {{
            {mh_curveValues}
        }}
    }}
}}'''

    return usda_str


#### ...

In [27]:
weights_mh_hat = regressor.predict(weights_blends)
weights_mh_hat.shape


weights_mh_diff = weights_mh_hat - weights_mh

n_cols = 74
# df_diff = pd.DataFrame(columns=[i for i in range(n_cols)], values=weights_mh_diff)
df_diff = pd.DataFrame(weights_mh_diff)
foo = df_diff.round(5)


# df = pd.DataFrame(columns=[i for i in range(n_cols)])
# for i in range(30):
#     df.loc[i] = weights_mh_diff[i, 0:n_cols]

# foo.iloc[:, 60:75].describe()
# foo.shape


,60,61,62,63,64,65,66,67,68,69,70,71,72,73
count,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0


---

In [50]:
n_frames, _ = weights_mh.shape

mh_curveValues = ''
for i in range(n_frames):
    # row_weights = ', '.join([str(w) for w in weights_mh[i, :]])
    row_weights = ', '.join([f'{w:.8f}' for w in weights_mh[i, :]])
    row = f'{i}: [{row_weights}],' 
    mh_curveValues += f'\n\t\t\t\t{row}'
    # if i > 5: break

usda_str = get_usda_str(
    prim_xform='World',
    prim_skew_anim='anim_canto_primeiro_usd',
    frame_rate=30,
    n_frames=n_frames,
    mh_curveValues=mh_curveValues
)

f = open(path_usda, 'w')
f.write(usda_str)
f.close()

## Exploration

#### aux

In [9]:

def intersection(list1: list, list2: list):
    lst3 = [value for value in list1 if value in list2]
    return lst3


#### Foo

In [19]:
n_mh_curves = len(names_mh_curves)
n_blends = len(names_blends)

matches = intersection(names_blends, name_mh_curves)
n_matches = len(matches)

print(f'n_mh_curves: {n_mh_curves}')
print(f'n_blends: {n_blends}')
print(f'n_matches: {n_matches}')


n_mh_curves: 74
n_blends: 46
n_matches: 0
